In [30]:
# get gps coordinates from geopy
import json

# import urlopen from urllib.request
from urllib.request import urlopen

# open following url to get ipaddress
urlopen("http://ipinfo.io/json")

# load data into array
data = json.load(urlopen("http://ipinfo.io/json"))

# extract lattitude
lat = data['loc'].split(',')[0]

# extract longitude
lon = data['loc'].split(',')[1]

print("Latitude:", lat)
print("Longitude:", lon)

Latitude: -23.5475
Longitude: -46.6361


In [37]:
# MEDIA DE PREÇOS DE TODOS OS DATAFRAMES
import json
import pandas as pd
import geopandas as gpd
import geopy
import matplotlib.pyplot as plt
import tqdm
from urllib.request import urlopen
from openpyxl import Workbook
from geopy.geocoders import Nominatim
from tqdm._tqdm_notebook import tqdm_notebook


def calcular_media_precos(datasets):
    produtos_precos = {}
    
    for dataset in datasets:
        df = pd.read_excel(dataset)
        for _, row in df.iterrows():
            produto = row['produtos']
            preco = row['preço']
            if produto not in produtos_precos:
                produtos_precos[produto] = []
            produtos_precos[produto].append(preco)
    
    media_precos = {}
    for produto, precos in produtos_precos.items():
        media_precos[produto] = sum(precos) / len(precos)
    
    return media_precos


# abre url para pegar o endereço do ip
urlopen("http://ipinfo.io/json")

# carrega os dados em um array
dado = json.load(urlopen("http://ipinfo.io/json"))

# traz latitude do json
lat = dado['loc'].split(',')[0]

# traz longitude do json
lon = dado['loc'].split(',')[1]


locator = Nominatim(user_agent="meuGeocoder")
coordinates = (lat + "," + lon)
location = locator.reverse(coordinates)


cidade = location.raw.get('address', {}).get('city')
condado = location.raw.get('address', {}).get('county')
estado = location.raw.get('address', {}).get('state')
regiao = location.raw.get('address', {}).get('region')


def regiao_cresce(media_precos):
    for produto in media_precos:
         media_precos[produto] *= 1.05


def regiao_diminui(media_precos):
    for produto in media_precos:
        media_precos[produto] *= 0.95


def main():
    datasets = ['dataset1.xlsx', 'dataset2.xlsx', 'dataset3.xlsx', 'dataset4.xlsx', 'dataset5.xlsx']
    media_precos = calcular_media_precos(datasets)

    if estado == "São Paulo":
        regiao_cresce(media_precos)

    # Criação do arquivo Excel
    wb = Workbook()
    ws = wb.active
    ws.append(['Produto', 'Preços', 'Estado'])

    for produto, media_preco in media_precos.items():
        ws.append([produto, round(media_preco, 2), estado])

    wb.save('preco_ideal.xlsx')
    print("Arquivo excel criado com êxito!")

    print("Cidade:", cidade)
    print("Condado:", condado)
    print("Estado:", estado)
    print("Região:", regiao)

if __name__ == "__main__":
    main()

Arquivo excel criado com êxito!
Cidade: São Paulo
Condado: Região Metropolitana de São Paulo
Estado: São Paulo
Região: Região Sudeste
